In [1]:
import pandas as pd
import json

import os

from elsapy.elsclient import ElsClient
from elsapy.elsprofile import ElsAuthor
from elsapy.elssearch import ElsSearch
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from google.oauth2 import service_account
from google.cloud import bigquery
import datetime

In [26]:
pip install pyarrow

Note: you may need to restart the kernel to use updated packages.


In [81]:
#API_KEY="640ababb3eaaa3723c7cc11be848250b"
API_KEY="ecfa002fa03d10079a37957428838986"
scopus_client = ElsClient(API_KEY)

In [28]:
def saveAuthorInfo(self,author_id,author_name,client):
    
    
    
    # Initialize author with uri
    my_auth = ElsAuthor(
        uri = f'https://api.elsevier.com/content/author/author_id/{author_id}')

    if my_auth.read(client):
        data = my_auth.data
        del data["@status"]
        del data["@_fa"]
        json_object = json.dumps(data)
        print(type(json_object))
        json_object = json.dumps(json_object)
        author_name = author_name.replace(" ","_")
        with open(f"{self.dag_path}/data/scopus/raw/authors_raw_data/{author_name}({author_id}).json", "w") as f:
            f.write(json_object)

In [ ]:
def getAuthorData(author_id,author_name,client):
    
    # Initialize author with uri
    my_auth = ElsAuthor(
        uri = f'https://api.elsevier.com/content/author/author_id/{author_id}')

    if my_auth.read(client):
        data = my_auth.data
        del data["@status"]
        del data["@_fa"]
        json_object = json.dumps(data)
        data = json.loads(json_object)
        data["Scopus_ID"]=author_id
        print(data)
        return data

In [29]:
authors_ids_data= pd.read_excel("/Users/juansebastianvargastorres/workflow/firstAirFlow/data/authors_ids/authors_ids.xlsx")
authors_ids_data=authors_ids_data.loc[authors_ids_data["Scopus_ID"].notna()]
authors_ids_data["Cedula"] = authors_ids_data["Cedula"].fillna(-1).astype(int).astype(str)
authors_ids_data["Scopus_ID"] = authors_ids_data["Scopus_ID"].astype(int).astype(str)

# print(authors_ids_data)

authors_data=[]

for index, row in authors_ids_data.iterrows():  
    authors_data.append(getAuthorData(author_id=row["Scopus_ID"],author_name=row["Nombre"],client=scopus_client))


{'coredata': {'prism:url': 'http://api.elsevier.com/content/author/author_id/24832026300', 'dc:identifier': 'AUTHOR_ID:24832026300', 'eid': '9-s2.0-24832026300', 'document-count': '63', 'cited-by-count': '370', 'citation-count': '392', 'link': [{'@href': 'https://www.scopus.com/authid/detail.uri?partnerID=HzOxMe3b&authorId=24832026300&origin=inward', '@rel': 'scopus-author', '@_fa': 'true'}, {'@href': 'http://api.elsevier.com/content/author/author_id/24832026300', '@rel': 'self', '@_fa': 'true'}, {'@href': 'http://api.elsevier.com/content/search/scopus?query=au-id%2824832026300%29', '@rel': 'search', '@_fa': 'true'}]}, 'affiliation-current': {'@id': '60033545', '@href': 'http://api.elsevier.com/content/affiliation/affiliation_id/60033545'}, 'affiliation-history': {'affiliation': [{'@_fa': 'true', '@id': '118807642', '@href': 'http://api.elsevier.com/content/affiliation/affiliation_id/118807642'}, {'@_fa': 'true', '@id': '60033545', '@href': 'http://api.elsevier.com/content/affiliation/

In [98]:
df = pd.DataFrame(authors_data)

In [99]:
df.head(5)

,coredata,affiliation-current,affiliation-history,subject-areas,author-profile,Scopus_ID
0,{'prism:url': 'http://api.elsevier.com/content...,"{'@id': '60033545', '@href': 'http://api.elsev...","{'affiliation': [{'@_fa': 'true', '@id': '1188...","{'subject-area': [{'@_fa': 'true', '@abbrev': ...","{'status': 'update', 'date-created': {'@day': ...",24832026300
1,{'prism:url': 'http://api.elsevier.com/content...,"{'@id': '106422639', '@href': 'http://api.else...","{'affiliation': [{'@_fa': 'true', '@id': '6003...","{'subject-area': [{'@_fa': 'true', '@abbrev': ...","{'status': 'update', 'date-created': {'@day': ...",57212998692
2,{'prism:url': 'http://api.elsevier.com/content...,"{'@id': '60033545', '@href': 'http://api.elsev...","{'affiliation': [{'@_fa': 'true', '@id': '6003...","{'subject-area': [{'@_fa': 'true', '@abbrev': ...","{'status': 'update', 'date-created': {'@day': ...",20436581100
3,{'prism:url': 'http://api.elsevier.com/content...,"{'@id': '60011966', '@href': 'http://api.elsev...","{'affiliation': [{'@_fa': 'true', '@id': '6001...","{'subject-area': [{'@_fa': 'true', '@abbrev': ...","{'status': 'update', 'date-created': {'@day': ...",7103099833
4,{'prism:url': 'http://api.elsevier.com/content...,"{'@id': '126688257', '@href': 'http://api.else...","{'affiliation': [{'@_fa': 'true', '@id': '1266...","{'subject-area': [{'@_fa': 'true', '@abbrev': ...","{'status': 'new', 'date-created': {'@day': '12...",57226563089


In [98]:
auth_products = ElsSearch("AU-ID({})".format("24832026300"), "scopus")
auth_products.execute(scopus_client)
json_object = json.dumps(auth_products.results)
dictionary = json.loads(json_object)

In [105]:
products=[]
for product in dictionary:
    # del product["link"]
    # del product["@_fa"]
    # del product["prism:url"]
    product["Scopus_ID"]=1213
    product["Author_name"]="hola"
    
    
    containsJaverianaAffil=False
    for affs in product["affiliation"]:
        if "javeriana" in str(affs["affilname"]).lower() or "hospital universitario san ignacio" in str(affs["affilname"]).lower():
            containsJaverianaAffil=True
            
    if containsJaverianaAffil:
        products.append(product)
    
    # if 'prism:isbn' in product:
    #     isbns = product.get('prism:isbn', [])
    #     product.update({f"isbn_{index+1}": str('{:.0f}'.format(int(isbn['$']))) for index, isbn in enumerate(isbns)})
    #     print("--------------")    
    
    print("-------------------")
#str('{:.0f}'.format(isbn['$'])

Pontificia Universidad Javeriana
-------------------
Pontificia Universidad Javeriana
-------------------
Pontificia Universidad Javeriana
Colombian Center of Excellence and Appropriation on Big Data and Data Analytics (CAOBA)
-------------------
Universidad de Jaén
-------------------
Pontificia Universidad Javeriana
-------------------
Universidad Tecnológica de Bolívar
Pontificia Universidad Javeriana
-------------------
Pontificia Universidad Javeriana
-------------------
Pontificia Universidad Javeriana
-------------------
Pontificia Universidad Javeriana
Center of Excellence and Appropriation in Big Data and Data Analytics (CAOBA)
-------------------
Pontificia Universidad Javeriana
-------------------
Pontificia Universidad Javeriana
-------------------
Pontificia Universidad Javeriana
-------------------
Pontificia Universidad Javeriana
-------------------
Pontificia Universidad Javeriana
Center of Excellence and Appropriation in Big Data and Data Analytics (CAOBA)
------------

In [96]:
products_df=pd.DataFrame(products)

In [85]:
pr_columns=products_df.columns
formatted_pr_columns = [str(col).replace("-", "_") for col in pr_columns]
formatted_pr_columns = [str(col).replace("dc:", "") for col in formatted_pr_columns]
formatted_pr_columns = [str(col).replace("dc:", "") for col in formatted_pr_columns]

In [86]:
rename_columns= dictionary = dict(zip(pr_columns, formatted_pr_columns))

In [87]:
rename_columns

{'dc:identifier': 'identifier',
 'eid': 'eid',
 'dc:title': 'title',
 'dc:creator': 'creator',
 'prism:publicationName': 'prism:publicationName',
 'prism:isbn': 'prism:isbn',
 'prism:pageRange': 'prism:pageRange',
 'prism:coverDate': 'prism:coverDate',
 'prism:coverDisplayDate': 'prism:coverDisplayDate',
 'prism:doi': 'prism:doi',
 'citedby-count': 'citedby_count',
 'affiliation': 'affiliation',
 'prism:aggregationType': 'prism:aggregationType',
 'subtype': 'subtype',
 'subtypeDescription': 'subtypeDescription',
 'source-id': 'source_id',
 'openaccess': 'openaccess',
 'openaccessFlag': 'openaccessFlag',
 'Scopus_ID': 'Scopus_ID',
 'Author_name': 'Author_name',
 'prism:eIssn': 'prism:eIssn',
 'prism:volume': 'prism:volume',
 'freetoread': 'freetoread',
 'freetoreadLabel': 'freetoreadLabel',
 'prism:issueIdentifier': 'prism:issueIdentifier',
 'article-number': 'article_number',
 'pubmed-id': 'pubmed_id',
 'prism:issn': 'prism:issn',
 'pii': 'pii'}

In [88]:
products_df=products_df.rename(columns=rename_columns)

In [89]:
products_df["identifier"]=products_df["identifier"].str.replace("SCOPUS_ID:","")

In [90]:
products_df

,identifier,eid,title,creator,prism:publicationName,prism:isbn,prism:pageRange,prism:coverDate,prism:coverDisplayDate,prism:doi,citedby_count,affiliation,prism:aggregationType,subtype,subtypeDescription,source_id,openaccess,openaccessFlag,Scopus_ID,Author_name,prism:eIssn,prism:volume,freetoread,freetoreadLabel,prism:issueIdentifier,article_number,pubmed_id,prism:issn,pii
0,85143694709,2-s2.0-85143694709,Analysis of social networks publications for s...,Bustos O.,2022 Congreso Internacional de Innovacion y Te...,"[{'@_fa': 'true', '$': '9781665465250'}]",None,2022-01-01,2022,10.1109/CONIITI57704.2022.9953669,0,"[{'@_fa': 'true', 'affilname': 'Pontificia Uni...",Conference Proceeding,cp,Conference Paper,21101125305,0,False,1213,hola,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,85141453261,2-s2.0-85141453261,Hybrid Onion Layered System for the Analysis o...,Moreno-Sandoval L.G.,IEEE Access,NaN,115435-115468,2022-01-01,2022,10.1109/ACCESS.2022.3217467,0,"[{'@_fa': 'true', 'affilname': 'Pontificia Uni...",Journal,ar,Article,21100374601,1,True,1213,hola,21693536,10,"{'value': [{'$': 'all'}, {'$': 'publisherfullg...","{'value': [{'$': 'All Open Access'}, {'$': 'Go...",NaN,NaN,NaN,NaN,NaN
2,85113437805,2-s2.0-85113437805,Celebrity profiling through linguistic analysi...,Moreno-Sandoval L.G.,Computational Social Networks,NaN,None,2021-12-01,December 2021,10.1186/s40649-021-00097-w,2,"[{'@_fa': 'true', 'affilname': 'Pontificia Uni...",Journal,ar,Article,21100870845,1,True,1213,hola,21974314,8,"{'value': [{'$': 'all'}, {'$': 'publisherhybri...","{'value': [{'$': 'All Open Access'}, {'$': 'Hy...",1,16,NaN,NaN,NaN
3,85105306278,2-s2.0-85105306278,Collecting specialty-related medical terms: De...,López-Úbeda P.,BMC Medical Informatics and Decision Making,NaN,None,2021-12-01,December 2021,10.1186/s12911-021-01495-w,1,"[{'@_fa': 'true', 'affilname': 'Universidad de...",Journal,ar,Article,23602,1,True,1213,hola,14726947,21,"{'value': [{'$': 'all'}, {'$': 'publisherfullg...","{'value': [{'$': 'All Open Access'}, {'$': 'Go...",1,145,33947365,NaN,NaN
4,85107238076,2-s2.0-85107238076,Transfer learning for classifying Spanish and ...,Pomares-Quimbaya A.,Public Health and Informatics: Proceedings of ...,"[{'@_fa': 'true', '$': '9781643681856'}, {'@_f...",377-381,2021-07-01,1 July 2021,10.3233/SHTI210184,1,"[{'@_fa': 'true', 'affilname': 'Pontificia Uni...",Book,ch,Book Chapter,21101061290,1,True,1213,hola,NaN,NaN,"{'value': [{'$': 'all'}, {'$': 'publisherhybri...","{'value': [{'$': 'All Open Access'}, {'$': 'Hy...",NaN,NaN,34042769,NaN,NaN
5,85100533991,2-s2.0-85100533991,Detection of Sociolinguistic Features in Digit...,Puertas E.,Cognitive Computation,NaN,518-537,2021-03-01,March 2021,10.1007/s12559-021-09818-9,3,"[{'@_fa': 'true', 'affilname': 'Universidad Te...",Journal,ar,Article,19400158515,0,False,1213,hola,18669964,13,NaN,NaN,2,NaN,NaN,18669956,NaN
6,85084329548,2-s2.0-85084329548,Stock market movement forecast: A Systematic r...,Bustos O.,Expert Systems with Applications,NaN,None,2020-10-15,15 October 2020,10.1016/j.eswa.2020.113464,87,"[{'@_fa': 'true', 'affilname': 'Pontificia Uni...",Journal,re,Review,24201,0,False,1213,hola,NaN,156,NaN,NaN,NaN,113464,NaN,09574174,S0957417420302888
7,85086910702,2-s2.0-85086910702,Leveraging PubMed to create a specialty-based ...,Pomares-Quimbaya A.,Studies in Health Technology and Informatics,"[{'@_fa': 'true', '$': '9781643680828'}]",292-296,2020-06-16,16 June 2020,10.3233/SHTI200169,1,"[{'@_fa': 'true', 'affilname': 'Pontificia Uni...",Book Series,cp,Conference Paper,19582,0,False,1213,hola,18798365,270,NaN,NaN,NaN,NaN,32570393,09269630,NaN
8,85121794424,2-s2.0-85121794424,"Assembly of polarity, emotion and user statist...",Moreno-Sandoval L.G.,CEUR Workshop Proceedings,NaN,None,2020-01-01,2020,NaN,0,"[{'@_fa': 'true', 'affilname': 'Pontificia Uni...",Conference Proceeding,cp,Conference Paper,21100218356,0,False,1213,hola,NaN,2696,NaN,NaN,NaN,NaN,NaN,16130073,NaN
9,85137544530,2-s2.0-85137544530,Concept att

In [2]:
core=pd.read_csv("/Users/juansebastianvargastorres/workflow/firstAirFlow/data/scopus/clean/clean_authors_core_data/core_data.csv")

In [3]:
core

,Scopus_ID,document-count,cited-by-count,citation-count,orcid,Author_name
0,24832026300,63,371,393,NaN,Pomares Quimbaya Alexandra
1,57212998692,58,546,596,0000-0002-9695-3775,Carlos Andres Parra Acevedo
2,20436581100,6,46,46,NaN,"Burbano Valente, Johanna"
3,7103099833,274,5675,8207,NaN,"Duran Echeverry, Alicia Del Socorro"
4,57226563089,1,0,0,NaN,"Escobar Altare, Andrea Paola"
5,57215079697,3,2,2,NaN,"Escobar Altare, Andrea Paola"
6,57216526998,1,1,1,NaN,"Escobar Altare, Andrea Paola"
7,24462330500,1,0,0,NaN,"Santacoloma Giraldo, Andres"
8,13403648000,1,2,2,NaN,"Garcia Padilla, Dennys del Rocio"
9,56491243500,4,19,19,NaN,"Rubio Leon, Diana Carolina"


In [7]:
credentials = service_account.Credentials.from_service_account_file(
            "/Users/juansebastianvargastorres/workflow/firstAirFlow/data/bigQuery/viceinvestigacion-7d7d86b22e96.json",
            scopes = ["https://www.googleapis.com/auth/cloud-platform"])
        
client = bigquery.Client(credentials = credentials, project = credentials.project_id)
        

In [5]:
# schema=[
#         bigquery.SchemaField("Scopus_id","INTEGER", mode="REQUIRED"),
#         bigquery.SchemaField("orcid", "STRING", mode="NULLABLE"),
#         bigquery.SchemaField("document_count", bigquery.enums.SqlTypeNames.INT64),
#         bigquery.SchemaField("cited_by_count", bigquery.enums.SqlTypeNames.INT64),
#         bigquery.SchemaField("citation_count", bigquery.enums.SqlTypeNames.INT64),
#         bigquery.SchemaField("pick_date", "TIMESTAMP", mode="REQUIRED")
#         ]

In [13]:
df_testing=pd.read_csv("/Users/juansebastianvargastorres/workflow/firstAirFlow/data/scopus/to_upload_data/core_data.csv")

In [14]:
df_testing=df_testing.rename(columns={"document-count":"document_count","cited-by-count":"cited_by_count","citation-count":"citation_count"})

In [43]:
df_testing.head(5)

,Scopus_ID,document_count,cited_by_count,citation_count,orcid,Author_name,pick_date
0,24832026300,63,371,393,NaN,Pomares Quimbaya Alexandra,2023-06-06 22:37:06.500517
1,57212998692,58,546,596,0000-0002-9695-3775,Carlos Andres Parra Acevedo,2023-06-06 22:37:06.500517
2,20436581100,6,46,46,NaN,"Burbano Valente, Johanna",2023-06-06 22:37:06.500517
3,7103099833,274,5675,8207,NaN,"Duran Echeverry, Alicia Del Socorro",2023-06-06 22:37:06.500517
4,57226563089,1,0,0,NaN,"Escobar Altare, Andrea Paola",2023-06-06 22:37:06.500517


In [28]:
dataset_id = 'socupus_data'
table_id = "core_data_test"
full_table_id = f'{client.project}.{dataset_id}.{table_id}' 
load_job_config = bigquery.LoadJobConfig(write_disposition = "WRITE_APPEND")    
job= client.load_table_from_dataframe(df_testing,full_table_id,job_config=load_job_config)
print(job.result())

NameError: name 'df_testing' is not defined

In [18]:
df_subject_test=pd.read_csv("/Users/juansebastianvargastorres/workflow/firstAirFlow/data/scopus/to_upload_data/subject_areas.csv")

In [44]:
df_subject_test.head(5)

,abbrev,Codigo,Subject_area,Scopus_ID,Author_name
0,MATH,2614,Theoretical Computer Science,24832026300,Pomares Quimbaya Alexandra
1,ENGI,2201,Engineering (miscellaneous),24832026300,Pomares Quimbaya Alexandra
2,SOCI,3304,Education,24832026300,Pomares Quimbaya Alexandra
3,MATH,2604,Applied Mathematics,24832026300,Pomares Quimbaya Alexandra
4,BUSI,1400,"Business, Management and Accounting (all)",24832026300,Pomares Quimbaya Alexandra


In [52]:
duplicated_rows = df_subject_test[df_subject_test.duplicated(subset=['Subject_area',"Scopus_ID"])]

In [53]:
duplicated_rows

,abbrev,Codigo,Subject_area,Scopus_ID,Author_name


In [35]:
(df_subject_test.loc[df_subject_test["Scopus_ID"] == 24832026300])

(42, 5)

In [37]:
(df_subject_test.loc[df_subject_test["Scopus_ID"] == 24832026300]).drop_duplicates(subset=['Subject_area']).shape

(42, 5)

In [39]:
df_subject_test.drop_duplicates(subset=['Subject_area',"Scopus_ID"]).shape

(301, 5)

In [19]:
institucional = pd.read_excel("/Users/juansebastianvargastorres/Documents/GitHub/Vic_ETL_airflow/data/repositorio_inst/repositorio_PUJ 02_05_2023.xlsx")

In [20]:
institucional.head(5)

,Autor - dc.contributor.author,Email - dc.creator.email,Director - dc.contributor.advisor,Evaluador - dc.contributor.evaluator,URI - dc.identifier.uri,Programa - dc.publisher.program,Facultad - dc.publisher.faculty,Título - dc.title.,Fecha,Año,Resumen - dc.description.abstract,Abstract English - dc.description.abstractenglish,Palabras Clave - dc.subject,Keywords - dc.subject.keyword,Temas - dc.subject.armarc
0,"Castañeda Acosta, Javier Alberto",javialbca7112@gmail.com,"Vivas Alban, María del Socorro","Gil Espinosa, María Isabel",http://hdl.handle.net/10554/64101,Teología,Facultad de Teología,Comunicación auténtica de la comunión eclesial...,2023-05-02,2023,Es urgente resignificar y retomar la experienc...,It is urgent to redefine and retake the relati...,Comunicación - Comunión - Encuentro - Persona ...,Communication - Communion - Meeting - Person -...,Trinidad - Comunión - Vida religiosa - Teolog...
1,"Florián Sánchez, Hans Leonardo",hans-florian@javeriana.edu.co,"Solarte Rodríguez, Mario Roberto","Caycedo Casallas, Carlos Bernardo - Louidor, W...",http://hdl.handle.net/10554/64102,Doctorado en Filosofía,Facultad de Filosofía,Derecho de hospitalidad: ¿respuesta cosmopolit...,2023-05-02,2023,Resolver la pregunta: ¿Cuáles han sido los apo...,In order to answer the question: What contribu...,Filosofía - Pedagogía - Migración - Hospitalid...,Philosophy - Pedagogy - Migration - Hospitalit...,Hospitalidad - Pedagogía - Internacionalismo ...
2,"Bejarano Puello, Rosa María",NaN,"Rueda Gómez, Alba Lucia",NaN,http://hdl.handle.net/10554/64100,Nutrición y Dietética,Facultad de Ciencias,Diseño de un programa de mejoramiento de la ca...,2023-04-28,2023,En este trabajo se documenta el estudio median...,This academic document shows the study and eva...,Máquinas dispensadoras saludables - Sistema de...,Healthy Vending Machine - Quality management s...,Administración de la calidad - Mejoramiento de...
3,"Peralta Bonilla, Berlly",berper81@hotmail.com,"Guáqueta Solórzano, Victoria Eugenia","Clavijo Ponce, Neidy Lorena",http://hdl.handle.net/10554/64098,Maestría en Gestión Ambiental,Facultad de Estudios Ambientales y Rurales,Conservación del páramo de Sumapaz: Identifica...,2023-04-28,2023,"En esta investigación, se exploran las práctic...","In this research, the conservation practices o...",Sumapaz - Medios de vida - Ganadería - Páramo ...,Sumapaz - Livelihoods - Livestock - Paramo - C...,Páramos - Colombia - Conservación del medio am...
4,"Rodríguez Angulo, Pablo",pablorodri99@gmail.com,"Gómez Pérez, Gustavo","Unger Parra, Biviana",http://hdl.handle.net/10554/64097,Filosofía,Facultad de Filosofía,Violencia Filosófica : sobre el Gorgias de Platón,2023-04-28,2023,La violencia filosófica es un tipo de violenci...,Philosophical violence is a type of violence c...,Violencia filosófica - Injusticia epistémica -...,Timaeus - Gorgias - Epistemic violence - María...,Dignidad - Justicia - Violencia - Aspectos fil...


In [21]:
pr_columns=institucional.columns
formatted_pr_columns = [str(col).replace(" - ", "_") for col in pr_columns]
formatted_pr_columns = [str(col).replace(".", "_") for col in formatted_pr_columns]
rename_columns= dictionary = dict(zip(pr_columns, formatted_pr_columns))

In [22]:
institucional=institucional.rename(columns=rename_columns)

In [23]:
institucional.head(5)

,Autor_dc_contributor_author,Email_dc_creator_email,Director_dc_contributor_advisor,Evaluador_dc_contributor_evaluator,URI_dc_identifier_uri,Programa_dc_publisher_program,Facultad_dc_publisher_faculty,Título_dc_title_,Fecha,Año,Resumen_dc_description_abstract,Abstract English_dc_description_abstractenglish,Palabras Clave_dc_subject,Keywords_dc_subject_keyword,Temas_dc_subject_armarc
0,"Castañeda Acosta, Javier Alberto",javialbca7112@gmail.com,"Vivas Alban, María del Socorro","Gil Espinosa, María Isabel",http://hdl.handle.net/10554/64101,Teología,Facultad de Teología,Comunicación auténtica de la comunión eclesial...,2023-05-02,2023,Es urgente resignificar y retomar la experienc...,It is urgent to redefine and retake the relati...,Comunicación - Comunión - Encuentro - Persona ...,Communication - Communion - Meeting - Person -...,Trinidad - Comunión - Vida religiosa - Teolog...
1,"Florián Sánchez, Hans Leonardo",hans-florian@javeriana.edu.co,"Solarte Rodríguez, Mario Roberto","Caycedo Casallas, Carlos Bernardo - Louidor, W...",http://hdl.handle.net/10554/64102,Doctorado en Filosofía,Facultad de Filosofía,Derecho de hospitalidad: ¿respuesta cosmopolit...,2023-05-02,2023,Resolver la pregunta: ¿Cuáles han sido los apo...,In order to answer the question: What contribu...,Filosofía - Pedagogía - Migración - Hospitalid...,Philosophy - Pedagogy - Migration - Hospitalit...,Hospitalidad - Pedagogía - Internacionalismo ...
2,"Bejarano Puello, Rosa María",NaN,"Rueda Gómez, Alba Lucia",NaN,http://hdl.handle.net/10554/64100,Nutrición y Dietética,Facultad de Ciencias,Diseño de un programa de mejoramiento de la ca...,2023-04-28,2023,En este trabajo se documenta el estudio median...,This academic document shows the study and eva...,Máquinas dispensadoras saludables - Sistema de...,Healthy Vending Machine - Quality management s...,Administración de la calidad - Mejoramiento de...
3,"Peralta Bonilla, Berlly",berper81@hotmail.com,"Guáqueta Solórzano, Victoria Eugenia","Clavijo Ponce, Neidy Lorena",http://hdl.handle.net/10554/64098,Maestría en Gestión Ambiental,Facultad de Estudios Ambientales y Rurales,Conservación del páramo de Sumapaz: Identifica...,2023-04-28,2023,"En esta investigación, se exploran las práctic...","In this research, the conservation practices o...",Sumapaz - Medios de vida - Ganadería - Páramo ...,Sumapaz - Livelihoods - Livestock - Paramo - C...,Páramos - Colombia - Conservación del medio am...
4,"Rodríguez Angulo, Pablo",pablorodri99@gmail.com,"Gómez Pérez, Gustavo","Unger Parra, Biviana",http://hdl.handle.net/10554/64097,Filosofía,Facultad de Filosofía,Violencia Filosófica : sobre el Gorgias de Platón,2023-04-28,2023,La violencia filosófica es un tipo de violenci...,Philosophical violence is a type of violence c...,Violencia filosófica - Injusticia epistémica -...,Timaeus - Gorgias - Epistemic violence - María...,Dignidad - Justicia - Violencia - Aspectos fil...


In [2]:
journals= pd.read_excel("/Users/juansebastianvargastorres/Documents/GitHub/Vic_ETL_airflow/data/journals/Historico de revistas 11 de Mayo 2023.xlsx")

In [3]:
journals.head(5)

,ISSN,Nombre revista,SJR Best Quartile,Country,Año2,Fuente,Lista SIR,Categoria,Vigencia Minciencias
0,14741741,Nature Reviews Immunology,Q1,United Kingdom,2018,SJR 2018,Scimago Journal Rank - SJR,NaN,NaN
1,17401534,Nature Reviews Microbiology,Q1,United Kingdom,2018,SJR 2018,Scimago Journal Rank - SJR,NaN,NaN
2,17483395,Nature Nanotechnology,Q1,United Kingdom,2018,SJR 2018,Scimago Journal Rank - SJR,NaN,NaN
3,17545706,Energy and Environmental Science,Q1,United Kingdom,2018,SJR 2018,Scimago Journal Rank - SJR,NaN,NaN
4,0147006X,Annual Review of Neuroscience,Q1,United States,2018,SJR 2018,Scimago Journal Rank - SJR,NaN,NaN


In [6]:
journals.head(5)

,ISSN,Name,SJR Best Quartile,Country,Year,Source,Lista SIR,Category,Vigencia Minciencias
0,14741741,Nature Reviews Immunology,Q1,United Kingdom,2018,SJR 2018,Scimago Journal Rank - SJR,NaN,NaN
1,17401534,Nature Reviews Microbiology,Q1,United Kingdom,2018,SJR 2018,Scimago Journal Rank - SJR,NaN,NaN
2,17483395,Nature Nanotechnology,Q1,United Kingdom,2018,SJR 2018,Scimago Journal Rank - SJR,NaN,NaN
3,17545706,Energy and Environmental Science,Q1,United Kingdom,2018,SJR 2018,Scimago Journal Rank - SJR,NaN,NaN
4,0147006X,Annual Review of Neuroscience,Q1,United States,2018,SJR 2018,Scimago Journal Rank - SJR,NaN,NaN


In [5]:
journals=journals.rename(columns={"Año2":"Year","Nombre revista":"Name","Fuente":"Source","Categoria":"Category"})

In [32]:
profesores = pd.read_excel("/Users/juansebastianvargastorres/Documents/GitHub/Vic_ETL_airflow/data/profesores_validados/Profesores_PUJ_27_de_Junio_2023_validado.xlsx")

In [33]:
pr_columns=profesores.columns
formatted_pr_columns = [str(col).replace(" ", "_") for col in pr_columns]

rename_columns= dictionary = dict(zip(pr_columns, formatted_pr_columns))
profesores=profesores.rename(columns=rename_columns)


In [34]:
profesores.head(10)

,Observaciones,Nombre,Nombre_Completo,Tipo_de_documento,ID_documento,CvLAC,CvLAC_con_comillas,Facultad,Programa,Número_de_programas,Año_de_inicio,SCOPUS_AUTHOR_ID,Contrato_Planta_Universidad_Javeriana,Contrato_Planta_Hospital_San_Igancio,Tipo,ID_Clínico
0,NaN,Juliana Atuesta Ortiz,"Atuesta Ortiz, Juliana",Cédula de ciudadanía,52867279,https://scienti.minciencias.gov.co/cvlac/visua...,"""https://scienti.minciencias.gov.co/cvlac/visu...",Facultad de Artes,Artes Escénicas,1,2020.0,NaN,NaN,NaN,NaN,NaN
1,NaN,José Alejandro Convers Elias,"Convers Elias, José Alejandro",Cédula de ciudadanía,79912950,https://scienti.minciencias.gov.co/cvlac/visua...,"""https://scienti.minciencias.gov.co/cvlac/visu...",Facultad de Artes,Artes Escénicas,1,2016.0,NaN,NaN,NaN,NaN,NaN
2,NaN,Catalina del Castillo Silva,"del Castillo Silva, Catalina",Cédula de ciudadanía,52713641,https://scienti.minciencias.gov.co/cvlac/visua...,"""https://scienti.minciencias.gov.co/cvlac/visu...",Facultad de Artes,Artes Escénicas,1,2012.0,NaN,NaN,NaN,NaN,NaN
3,NaN,Sara Regina Fonseca García,"Fonseca García, Sara Regina",Cédula de ciudadanía,37745136,https://scienti.minciencias.gov.co/cvlac/visua...,"""https://scienti.minciencias.gov.co/cvlac/visu...",Facultad de Artes,Artes Escénicas,1,2016.0,NaN,NaN,NaN,NaN,NaN
4,NaN,Eloisa Jaramillo Arango,"Jaramillo Arango, Eloisa",Cédula de ciudadanía,52849145,https://scienti.minciencias.gov.co/cvlac/visua...,"""https://scienti.minciencias.gov.co/cvlac/visu...",Facultad de Artes,Artes Escénicas,1,2019.0,NaN,NaN,NaN,NaN,NaN
5,NaN,Jhony Alexander Muñoz Agulera,"Muñoz Agulera, Jhony Alexander",Cédula de ciudadanía,9801590,https://scienti.minciencias.gov.co/cvlac/visua...,"""https://scienti.minciencias.gov.co/cvlac/visu...",Facultad de Artes,Artes Escénicas,1,2021.0,NaN,NaN,NaN,NaN,NaN
6,NaN,Victor Alfonso Quesada Aguilar,"Quesada Aguilar, Victor Alfonso",Cédula de ciudadanía,80797754,https://scienti.minciencias.gov.co/cvlac/visua...,"""https://scienti.minciencias.gov.co/cvlac/visu...",Facultad de Artes,Artes Escénicas,1,2017.0,NaN,NaN,NaN,NaN,NaN
7,NaN,Emilsen Rincón Vargas,"Rincón Vargas, Emilsen",Cédula de ciudadanía,51948441,https://scienti.minciencias.gov.co/cvlac/visua...,"""https://scienti.minciencias.gov.co/cvlac/visu...",Facultad de Artes,Artes Escénicas,1,2017.0,NaN,NaN,NaN,NaN,NaN
8,NaN,Jose Andres Areiza Padilla,"Areiza Padilla, Jose Andres",Cédula de ciudadanía,80099932,https://scienti.minciencias.gov.co/cvlac/visua...,"""https://scienti.minciencias.gov.co/cvlac/visu...",Facultad de Ciencias Económicas y Administrativas,Maestría en Administración,1,NaN,https://www.scopus.com/authid/detail.uri?autho...,NaN,NaN,NaN,NaN
9,NaN,Jaime Andres Bayona Bohorquez,"Bayona Bohorquez, Jaime Andres",Cédula de ciudadanía,80019278,https://scienti.minciencias.gov.co/cvlac/visua...,"""https://scienti.minciencias.gov.co/cvlac/visu...",Facultad de Ciencias Económicas y Administrativas,Maestría en Administración,1,NaN,https://www.scopus.com/authid/detail.uri?autho...,NaN,NaN,NaN,NaN


In [63]:
produccion = pd.read_excel("/Users/juansebastianvargastorres/Documents/GitHub/Vic_ETL_airflow/data/produccion_validada/Producción al 21 de Junio 2023.xlsx")
pr_columns=produccion.columns
formatted_pr_columns = [str(col).replace(" ", "_") for col in pr_columns]
formatted_pr_columns = [str(col).replace("  ", "_") for col in formatted_pr_columns]
formatted_pr_columns = [str(col).replace("á", "a") for col in formatted_pr_columns]
formatted_pr_columns = [str(col).replace("é", "e") for col in formatted_pr_columns]
formatted_pr_columns = [str(col).replace("í", "i") for col in formatted_pr_columns]
formatted_pr_columns = [str(col).replace("ó", "o") for col in formatted_pr_columns]
formatted_pr_columns = [str(col).replace("ñ", "nn") for col in formatted_pr_columns]
formatted_pr_columns = [str(col).replace("/", "_") for col in formatted_pr_columns]
rename_columns = dict(zip(pr_columns, formatted_pr_columns))
produccion=produccion.rename(columns=rename_columns)
produccion.loc[(produccion["Tipo_documental"]=="Artículo"),"Tipo_documental"]="Articulo"
produccion.loc[(produccion["Tipo_documental"]=="Article"),"Tipo_documental"]="Articulo"
produccion.loc[(produccion["Tipo_documental"]=="Book Chapter"),"Tipo_documental"]="Capitulo de Libro"
produccion.loc[(produccion["Tipo_documental"]=="Capítulo de libro"),"Tipo_documental"]="Capitulo de Libro"
produccion.loc[(produccion["Tipo_documental"]=="Capitulo de libro"),"Tipo_documental"]="Capitulo de Libro"
produccion.loc[(produccion["Tipo_documental"]=="Capítulo de Libro"),"Tipo_documental"]="Capitulo de Libro"

In [64]:
produccion["Tipo_documental"].unique()

array(['Acompañamiento de trabajo de grado', 'Articulo',
       'Capitulo de Libro', 'Libro', 'Conference Paper',
       'Meeting Abstract'], dtype=object)

In [56]:
produccion.head(5)

,Programa_de_creacion,Fecha_de_creacion,Cedula,Investigador_docente,Tipo_documental,Anno,Nombre_producto,Abstract,Key_words_thesaurus,DOI,ISSN_ISBN,Nombre_revista_libro,Referencia_bibliografica,Fuente,Coautores,Afiliacion_Coautores,Pais_coautores,Financiadores,#_citas_Scopus,Afiliacion_Citantes_Scopus,Pais_citantes_Scopus,#_citas_WoS,Afiliacion_Citantes_WoS,Pais_citantes_WoS,#_citas_Google_Scholar,Url_citantes_en_Google_Scholar,Afiliacion_Citantes_Google_Scholar
0,Pregrado en Ciencia de la Información,2023-06-20,12199588,"Cuellar Montealegre, Mario Fernando",Acompañamiento de trabajo de grado,2019,Imaginarios y desafíos del profesional en cien...,"El presente trabajo, tiene como finalidad dete...",Ciencia de la información - Profesional de la ...,NaN,NaN,NaN,"Nieves Benavides, Lizt Katerine - Sarmiento To...",Repositorio,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Pregrado en Ciencia de la Información,2023-06-20,12199588,"Cuellar Montealegre, Mario Fernando",Acompañamiento de trabajo de grado,2020,Pautas para la valoración estética de los bien...,Este trabajo se propone la comprensión del cri...,Bienes de interés cultural de carácter documen...,NaN,NaN,NaN,"López Mojica, Gloria Milena (2020) Pautas para...",Repositorio,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Pregrado en Ciencia de la Información,2023-06-20,12199588,"Cuellar Montealegre, Mario Fernando",Acompañamiento de trabajo de grado,2020,Sistematización de experiencias y aprendizajes...,Este trabajo de grado es una apuesta por siste...,Librerías - Librero - Sistematización de exper...,NaN,NaN,NaN,"García Hernández, Arturo (2020) Sistematizació...",Repositorio,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Pregrado en Ciencia de la Información,2023-06-20,12199588,"Cuellar Montealegre, Mario Fernando",Acompañamiento de trabajo de grado,2021,Lineamientos para una política institucional d...,El presente proyecto hace referencia al anális...,Universidad Pedagógica Nacional (Colombia) - R...,NaN,NaN,NaN,"Cuastuza Arango, Melissa Giselle (2021) Lineam...",Repositorio,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Pregrado en Ciencia de la Información,2023-06-20,12199588,"Cuellar Montealegre, Mario Fernando",Acompañamiento de trabajo de grado,2021,Propuesta para un modelo de gestión de conocim...,Este proyecto de investigación plantea una pro...,Modelos de gestión de conocimiento - Bibliotec...,NaN,NaN,NaN,"García Rojas, Deissy Jazmín - López Rangel, An...",Repositorio,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [57]:
credentials = service_account.Credentials.from_service_account_file(
           "/Users/juansebastianvargastorres/workflow/firstAirFlow/data/bigQuery/viceinvestigacion-7d7d86b22e96.json",
            scopes = ["https://www.googleapis.com/auth/cloud-platform"])
        
client = bigquery.Client(credentials = credentials, project = credentials.project_id)

In [58]:
dataset_id = 'socupus_data'
table_id = "produccion_validada_test"
full_table_id = f'{client.project}.{dataset_id}.{table_id}' 
load_job_config = bigquery.LoadJobConfig(write_disposition = "WRITE_APPEND")    
job= client.load_table_from_dataframe(produccion,full_table_id,job_config=load_job_config)
print(job.result())

LoadJob<project=viceinvestigacion, location=US, id=f666f4b3-5d9b-4888-890b-7698d2210307>


In [ ]:
journals= pd.read_csv("/Users/juansebastianvargastorres/Documents/GitHub/Vic_ETL_airflow/data/to_upload/journals.xlsx")

In [10]:
journals.head(5)

,ISSN,Name,SJR_Best_Quartile,Country,Year,Source,Lista_SIR,Category,Vigencia_Minciencias
0,14741741,Nature Reviews Immunology,Q1,United Kingdom,2018,SJR 2018,Scimago Journal Rank - SJR,NaN,NaN
1,17401534,Nature Reviews Microbiology,Q1,United Kingdom,2018,SJR 2018,Scimago Journal Rank - SJR,NaN,NaN
2,17483395,Nature Nanotechnology,Q1,United Kingdom,2018,SJR 2018,Scimago Journal Rank - SJR,NaN,NaN
3,17545706,Energy and Environmental Science,Q1,United Kingdom,2018,SJR 2018,Scimago Journal Rank - SJR,NaN,NaN
4,0147006X,Annual Review of Neuroscience,Q1,United States,2018,SJR 2018,Scimago Journal Rank - SJR,NaN,NaN


In [9]:
jn_columns=journals.columns
formatted_jn_columns = [str(col).replace(" ", "_") for col in jn_columns]
rename_columns = dict(zip(jn_columns, formatted_jn_columns))
journals=journals.rename(columns=rename_columns)

In [47]:
proyectos = pd.read_excel('/Users/juansebastianvargastorres/Documents/GitHub/Vic_ETL_airflow/data/proyectos/proyectos_template.xlsx')
py_columns=proyectos.columns
formatted_py_columns = [str(col).replace(" ", "_") for col in py_columns]
formatted_py_columns = [str(col).replace(".", "_") for col in formatted_py_columns]
rename_columns = dict(zip(py_columns, formatted_py_columns))
proyectos=proyectos.rename(columns=rename_columns)

In [48]:
proyectos.head(5)

,ID_PROPUESTA,TIPO_DE_PROPUESTA,FECHA_DE_NEGOCIACION,ID_PROYECTO,TITULO_DEL_PROYECTO,FECHA_INICIAL_REAL,FECHA_FINAL_REAL,NOMBRE_FACULTAD,NOMBRE_DEL_DEPARTAMENTO,CONVOCATORIA,NOMBRE_PATROCINADOR,TIPO_DE_FINANCIADOR,NOMBRE_GRUPO_DE_INVESTIGACION,MIEMBRO_DEL_EQUIPO,TIPO_DE_DOCUMENTO,NUMERO_DE_DOCUMENTO,NOMBRES_Y_APELLIDOS,ROL_EN_EL_PROYECTO,PORCENTAJE_DEDICACION,CÓDIGO_PRESUPUESTO,CÓDIGO_CONTRATO_SECRE_JURÍDICA,VALOR_APROBADO_PATROCINADOR,CONTRAPARTIDA_TERCEROS,CONTRAPARTIDA_REC__PROPI,CONTRAPARTIDA_REC__NUEVO,REC__NUEVOS_UNIDADE_ACADEMICA,VALOR_CONTRAPARTIDA_TOTAL,ESTADO
0,20000,Proyecto de investigación interno,2019,20000,Determinación de la respuesta de gel dosimétri...,2019-11-05 00:00:00,2022-02-28 00:00:00,Facultad de Ciencias,Dpto. de Física,VRI04-19,PONTIFICIA UNIVERSIDAD JAVERIANA,PUJ - Convocatorias,,"""00020394836""","""CC""","""1010169002""","""Andrea Johana Abril Fajardo""",Investigador principal,10.0,120130O0401200,NaN,50000000,0.00,31423755.99,0.0,0,0,NaN
1,20000,Proyecto de investigación interno,2019,20000,Determinación de la respuesta de gel dosimétri...,2019-11-05 00:00:00,2022-02-28 00:00:00,\N,\N,VRI04-19,PONTIFICIA UNIVERSIDAD JAVERIANA,PUJ - Convocatorias,,\N,\N,\N,"""Daniel Marlow""",Asesor,NaN,120130O0401200,NaN,50000000,0.00,31423755.99,0.0,0,0,NaN
2,20000,Proyecto de investigación interno,2019,20000,Determinación de la respuesta de gel dosimétri...,2019-11-05 00:00:00,2022-02-28 00:00:00,\N,\N,VRI04-19,PONTIFICIA UNIVERSIDAD JAVERIANA,PUJ - Convocatorias,,\N,\N,\N,"""C\u00e9sar Sierra""",Asesor,NaN,120130O0401200,NaN,50000000,0.00,31423755.99,0.0,0,0,NaN
3,20000,Proyecto de investigación interno,2019,20000,Determinación de la respuesta de gel dosimétri...,2019-11-05 00:00:00,2022-02-28 00:00:00,\N,\N,VRI04-19,PONTIFICIA UNIVERSIDAD JAVERIANA,PUJ - Convocatorias,,"""00010115666""","""CE""","""353873""","""Luis Agulles Pedros""",Asesor,NaN,120130O0401200,NaN,50000000,0.00,31423755.99,0.0,0,0,NaN
4,20001,Proyecto de investigación interno,2019,20001,Desarrollo de una herramienta informática para...,2020-04-01 00:00:00,2021-12-15 00:00:00,Fac. de Arquitectura y Diseño,Dpto. de Arquitectura,VRI04-19,PONTIFICIA UNIVERSIDAD JAVERIANA,PUJ - Convocatorias,Calidad y habitabilidad de la vivienda,"""00020313829""","""CC""","""80136645""","""Juan Guillermo Yunda Lozano""",Investigador principal,50.0,120132M0401200,NaN,49709537.13,5380000.04,74499933.39,0.0,0,0,NaN


In [9]:
data_types = {'EMPLID': str}
profesores_hijos = pd.read_excel("/Users/juansebastianvargastorres/Documents/GitHub/Vic_ETL_airflow/data/profesores_data/profesores_hijos_template.xlsx",dtype=data_types)
pr_h_columns=profesores_hijos.columns
formatted_pr_h_columns = [str(col).replace(" ", "_") for col in pr_h_columns]
formatted_pr_h_columns = [str(col).replace(".", "_") for col in formatted_pr_h_columns]
rename_columns = dict(zip(pr_h_columns, formatted_pr_h_columns))
profesores_hijos=profesores_hijos.rename(columns=rename_columns)
profesores_hijos["ETL_upload_date"] = datetime.datetime.now()

In [10]:
profesores_hijos.head(5)

,GP_PAYGROUP,EMPLID,NOMBRE_EMPLEADO,T_D_,DOCUMENTO,BUSINESS_UNIT,UNIDAD_DE_NEGOCIO,DEPTID,DEPARTAMENTO,REL_FAM,GENERO_HIJO,NOMBRE_HIJO,FECHA_NACIMIENTO_HIJO,EDAD_HIJO,Estado_Civil,Sexo,Horas,Edad,ETL_upload_date
0,GPB_PL_TEM,10115208,Jose Ricardo Acero Montañez,CC,4168985,BFTEO,BFac Teología,2430801000,Centro De Formación Teológica,Hijo (a),M,"ACERO SALGUERO,JUAN DIEGO",2016-08-01,6.92,Casado (a),M,40,49,2023-07-26 10:46:46.467660
1,GPB_PL_TEM,32636,Juan Camilo Acosta Mejia,CC,94063349,BFING,BFac Ingeniería,2180240000,Dpto. de Ingenieria Industrial,Hijo (a),M,"ACOSTA GOMEZ,PEDRO",2021-03-17,2.29,Casado (a),M,40,39,2023-07-26 10:46:46.467660
2,GPB_PL_TEM,32636,Juan Camilo Acosta Mejia,CC,94063349,BFING,BFac Ingeniería,2180240000,Dpto. de Ingenieria Industrial,Hijo (a),F,"ACOSTA GOMEZ,SUSANA",2022-11-16,0.62,Casado (a),M,40,39,2023-07-26 10:46:46.467660
3,GPB_PL_TEM,10051293,Margarita Maria Quiroga Cardenas,CC,51625455,BFODO,BFac Odontología,2300215000,Dpto. de Sistema Craneofacial,Hijo (a),F,"ACOSTA QUIROGA,LAURA MARGARITA",1990-11-10,32.66,Divorciado (a),F,35,61,2023-07-26 10:46:46.467660
4,GPB_PL_TEM,10051293,Margarita Maria Quiroga Cardenas,CC,51625455,BFODO,BFac Odontología,2300215000,Dpto. de Sistema Craneofacial,Hijo (a),M,"ACOSTA QUIROGA,SANTIAGO CAMILO",1996-05-07,27.16,Divorciado (a),F,35,61,2023-07-26 10:46:46.467660


In [38]:
data_types = {'EMPLID': str}
profesores_formacion = pd.read_excel("/Users/juansebastianvargastorres/Documents/GitHub/Vic_ETL_airflow/data/profesores_data/profesores_formacion_template.xlsx", dtype=data_types)
profesores_formacion["EMPLID"]= "ID"+profesores_formacion["EMPLID"]
profesores_formacion["ETL_upload_date"] = datetime.datetime.now()

In [39]:
profesores_formacion.head(5)

,EMPLID,NATIONAL_ID_TYPE,NATIONAL_ID,EFFDT,EFFSEQ,UJ_FORM_ACAD,UJ_INSTITUCION,SCHOOL_CODE,UJ_NOM_PROGRAMA,UJ_CONVALIDACION,UJ_RP_TIT_OBTENIDO,COUNTRY,STATE,UJ_FECHA_GRAD,DESCR60,UJ_NIV_IMPORT,DESCR_N_IMPT,M_HE,RN_MAX_HE,LAST_HE,ETL_upload_date
0,ID0134724,CC,38602285,2021-11-02,1,ESME,Universidad de Panamá,GENERADO,ESPECIALIZACIÓN EN GINECOLOGÍA Y OBSTETRICIA,N,Especialista en Ginecología y Obstetricia,PAN,,2014-10-15,Especialización Clínica Médica,11.0,Especialización Médico Quirúrgica,11.0,4,11.0,2023-07-26 11:22:55.036449
1,ID0134724,CC,38602285,2021-11-02,1,PREG,Universidad Libre,GENERADO,MEDICINA,N,Médico cirujano,COL,VAL,2007-04-20,Pregrado,7.0,Universitaria,11.0,2,7.0,2023-07-26 11:22:55.036449
2,ID0000000,CC,0,1900-01-01,1,ESPE,UNIVERSIDAD DE LOS ANDES,,ESPECIALISTA EN BANCA,S,ESPECIALISTA EN BANCA,,,1995-12-01,Especialización,4.0,Especialización Universitaria,4.0,7,4.0,2023-07-26 11:22:55.036449
3,ID0000000,CC,0,1900-01-01,1,MAES,UNIVERSIDAD DE LA AMISTAD DE LOS PUEBLOS,,MAGISTER EN ECONOMIA,S,MAGISTER EN ECONOMIA,,,1988-02-01,Maestría,3.0,Maestría,3.0,10,3.0,2023-07-26 11:22:55.036449
4,ID0000000,CC,0,1900-01-01,1,PREG,UNIVERSIDAD DE LA AMISTAD DE LOS PUEBLOS,,ECOMONIA,S,ECONOMISTA,,,1986-12-01,Pregrado,7.0,Universitaria,3.0,2,7.0,2023-07-26 11:22:55.036449


In [35]:
profesores_formacion_csv = pd.read_csv("/Users/juansebastianvargastorres/Documents/GitHub/Vic_ETL_airflow/data/to_upload/profesores_formacion.csv",low_memory=False)

In [36]:
profesores_formacion_csv.head(5)

,EMPLID,NATIONAL_ID_TYPE,NATIONAL_ID,EFFDT,EFFSEQ,UJ_FORM_ACAD,UJ_INSTITUCION,SCHOOL_CODE,UJ_NOM_PROGRAMA,UJ_CONVALIDACION,UJ_RP_TIT_OBTENIDO,COUNTRY,STATE,UJ_FECHA_GRAD,DESCR60,UJ_NIV_IMPORT,DESCR_N_IMPT,M_HE,RN_MAX_HE,LAST_HE,ETL_upload_date
0,134724.0,CC,38602285,2021-11-02,1,ESME,Universidad de Panamá,GENERADO,ESPECIALIZACIÓN EN GINECOLOGÍA Y OBSTETRICIA,N,Especialista en Ginecología y Obstetricia,PAN,,2014-10-15,Especialización Clínica Médica,11.0,Especialización Médico Quirúrgica,11.0,4,11.0,2023-07-26 16:05:28.747446
1,134724.0,CC,38602285,2021-11-02,1,PREG,Universidad Libre,GENERADO,MEDICINA,N,Médico cirujano,COL,VAL,2007-04-20,Pregrado,7.0,Universitaria,11.0,2,7.0,2023-07-26 16:05:28.747446
2,0.0,CC,0,1900-01-01,1,ESPE,UNIVERSIDAD DE LOS ANDES,,ESPECIALISTA EN BANCA,S,ESPECIALISTA EN BANCA,,,1995-12-01,Especialización,4.0,Especialización Universitaria,4.0,7,4.0,2023-07-26 16:05:28.747446
3,0.0,CC,0,1900-01-01,1,MAES,UNIVERSIDAD DE LA AMISTAD DE LOS PUEBLOS,,MAGISTER EN ECONOMIA,S,MAGISTER EN ECONOMIA,,,1988-02-01,Maestría,3.0,Maestría,3.0,10,3.0,2023-07-26 16:05:28.747446
4,0.0,CC,0,1900-01-01,1,PREG,UNIVERSIDAD DE LA AMISTAD DE LOS PUEBLOS,,ECOMONIA,S,ECONOMISTA,,,1986-12-01,Pregrado,7.0,Universitaria,3.0,2,7.0,2023-07-26 16:05:28.747446


In [40]:
semilleros = pd.read_csv("/Users/juansebastianvargastorres/Documents/GitHub/Vic_ETL_airflow/data/to_upload/semilleros.csv")

In [41]:
semilleros.head(5)

,Estado,Nombre_semillero,Facultad___Instituto,Departamento___Instituto,Area_de_conocimiento,Tutor,Correo_Tutor,Fecha_de_creacion_(Fecha_del_acta_en_el_que_fue_aprobado)
0,Activo,“GEMASA” - Generadores de Cuidado Humanizado e...,Fac. de Enfermería,Dpto. Enfermería de Salud Cole,Ciencias médicas y de la salud,Jenny Amparo Lozano Beltrán,lojenny@javeriana.edu.co,2020-07-24
1,Activo,Aberración cromática,Fac. de Artes,Dpto. de Artes visuales,Artes y humanidades,Yomayra Puentes,ypuentes@javeriana.edu.co,2019-03-22
2,Activo,Actividad Física Salud y Deporte,Fac. de Educación,Dpto. de Formación,Ciencias sociales,Marlucio De Souza Martins,mdesouzamartins@javeriana.edu.co,2015-05-07
3,Activo,Agricultura Biológica,Fac. de Ciencias,Dpto. de Biología,Ciencias agrícolas,Lucia Ana Patricia Diaz Ariza,luciaana@javeriana.edu.co,2015-04-24
4,Activo,"Agua para el futuro: monitoreo, modelación y g...",Fac. de Ingeniería,Dpto. de Civil,Ciencias naturales,Andrés Vargas Luna,avargasl@javeriana.edu.co,2019-11-12


In [ ]:
dataset_id = 'socupus_data'
table_id = "semilleros_test"
full_table_id = f'{client.project}.{dataset_id}.{table_id}' 
load_job_config = bigquery.LoadJobConfig(write_disposition = "WRITE_APPEND")    
job= client.load_table_from_dataframe(semilleros,full_table_id,job_config=load_job_config)
print(job.result())

In [120]:
proyectos= pd.read_excel("/Users/juansebastianvargastorres/Documents/GitHub/Vic_ETL_airflow/data/proyectos/proyectos_template.xlsx")

In [121]:
proyectos.head(5)

,ID_PROPUESTA,TIPO DE PROPUESTA,FECHA DE NEGOCIACION,ID PROYECTO,TITULO DEL PROYECTO,FECHA INICIAL REAL,FECHA FINAL REAL,NOMBRE FACULTAD,NOMBRE DEL DEPARTAMENTO,CONVOCATORIA,NOMBRE PATROCINADOR,TIPO DE FINANCIADOR,NOMBRE GRUPO DE INVESTIGACION,MIEMBRO DEL EQUIPO,TIPO DE DOCUMENTO,NUMERO DE DOCUMENTO,NOMBRES Y APELLIDOS,ROL EN EL PROYECTO,PORCENTAJE DEDICACION,CÓDIGO PRESUPUESTO,CÓDIGO CONTRATO SECRE JURÍDICA,VALOR APROBADO PATROCINADOR,CONTRAPARTIDA TERCEROS,CONTRAPARTIDA REC. PROPI,CONTRAPARTIDA REC. NUEVO,REC. NUEVOS UNIDADE ACADEMICA,VALOR CONTRAPARTIDA TOTAL,ESTADO
0,20000,Proyecto de investigación interno,2019,20000,Determinación de la respuesta de gel dosimétri...,2019-11-05 00:00:00,2022-02-28 00:00:00,Facultad de Ciencias,Dpto. de Física,VRI04-19,PONTIFICIA UNIVERSIDAD JAVERIANA,PUJ - Convocatorias,,"""00020394836""","""CC""","""1010169002""","""Andrea Johana Abril Fajardo""",Investigador principal,10.0,120130O0401200,NaN,50000000,0.00,31423755.99,0.0,0,0,NaN
1,20000,Proyecto de investigación interno,2019,20000,Determinación de la respuesta de gel dosimétri...,2019-11-05 00:00:00,2022-02-28 00:00:00,\N,\N,VRI04-19,PONTIFICIA UNIVERSIDAD JAVERIANA,PUJ - Convocatorias,,\N,\N,\N,"""Daniel Marlow""",Asesor,NaN,120130O0401200,NaN,50000000,0.00,31423755.99,0.0,0,0,NaN
2,20000,Proyecto de investigación interno,2019,20000,Determinación de la respuesta de gel dosimétri...,2019-11-05 00:00:00,2022-02-28 00:00:00,\N,\N,VRI04-19,PONTIFICIA UNIVERSIDAD JAVERIANA,PUJ - Convocatorias,,\N,\N,\N,"""C\u00e9sar Sierra""",Asesor,NaN,120130O0401200,NaN,50000000,0.00,31423755.99,0.0,0,0,NaN
3,20000,Proyecto de investigación interno,2019,20000,Determinación de la respuesta de gel dosimétri...,2019-11-05 00:00:00,2022-02-28 00:00:00,\N,\N,VRI04-19,PONTIFICIA UNIVERSIDAD JAVERIANA,PUJ - Convocatorias,,"""00010115666""","""CE""","""353873""","""Luis Agulles Pedros""",Asesor,NaN,120130O0401200,NaN,50000000,0.00,31423755.99,0.0,0,0,NaN
4,20001,Proyecto de investigación interno,2019,20001,Desarrollo de una herramienta informática para...,2020-04-01 00:00:00,2021-12-15 00:00:00,Fac. de Arquitectura y Diseño,Dpto. de Arquitectura,VRI04-19,PONTIFICIA UNIVERSIDAD JAVERIANA,PUJ - Convocatorias,Calidad y habitabilidad de la vivienda,"""00020313829""","""CC""","""80136645""","""Juan Guillermo Yunda Lozano""",Investigador principal,50.0,120132M0401200,NaN,49709537.13,5380000.04,74499933.39,0.0,0,0,NaN


In [122]:
# Assuming 'proyectos' is the DataFrame and 'FECHA DE NEGOCIACION' is the column containing the dates
def convert_to_datetime(value):
    if isinstance(value, int):
        # If the value is an integer (year), convert it to a datetime object
        return pd.to_datetime(str(value), format='%Y')
    else:
        # Otherwise, convert the value using pd.to_datetime
        return pd.to_datetime(value, errors='coerce')

proyectos['FECHA DE NEGOCIACION'] = proyectos['FECHA DE NEGOCIACION'].apply(lambda x: convert_to_datetime(x))


In [118]:
proyectos['FECHA DE NEGOCIACION'] = proyectos['FECHA DE NEGOCIACION'].apply(convert_to_datetime)

In [105]:
proyectos['FECHA DE NEGOCIACION'] = pd.to_datetime(proyectos['FECHA DE NEGOCIACION'], errors='coerce')

In [128]:
proyectos["FECHA DE NEGOCIACION"].unique()

<DatetimeArray>
['2019-01-01 00:00:00', '2020-01-01 00:00:00', '2021-01-01 00:00:00',
 '2022-01-01 00:00:00', '1990-05-22 00:00:00', '1990-12-26 00:00:00',
 '1993-02-01 00:00:00', '1993-06-01 00:00:00', '1993-08-31 00:00:00',
 '1993-10-27 00:00:00',
 ...
 '2023-03-08 00:00:00', '2023-03-10 00:00:00', '2023-03-16 00:00:00',
 '2023-04-01 00:00:00', '2023-04-12 00:00:00', '2023-04-24 00:00:00',
 '2023-05-01 00:00:00', '2023-05-15 00:00:00', '2023-05-16 00:00:00',
 '2023-11-20 00:00:00']
Length: 2025, dtype: datetime64[ns]

In [127]:
proyectos.head(5)

,ID_PROPUESTA,TIPO DE PROPUESTA,FECHA DE NEGOCIACION,ID PROYECTO,TITULO DEL PROYECTO,FECHA INICIAL REAL,FECHA FINAL REAL,NOMBRE FACULTAD,NOMBRE DEL DEPARTAMENTO,CONVOCATORIA,NOMBRE PATROCINADOR,TIPO DE FINANCIADOR,NOMBRE GRUPO DE INVESTIGACION,MIEMBRO DEL EQUIPO,TIPO DE DOCUMENTO,NUMERO DE DOCUMENTO,NOMBRES Y APELLIDOS,ROL EN EL PROYECTO,PORCENTAJE DEDICACION,CÓDIGO PRESUPUESTO,CÓDIGO CONTRATO SECRE JURÍDICA,VALOR APROBADO PATROCINADOR,CONTRAPARTIDA TERCEROS,CONTRAPARTIDA REC. PROPI,CONTRAPARTIDA REC. NUEVO,REC. NUEVOS UNIDADE ACADEMICA,VALOR CONTRAPARTIDA TOTAL,ESTADO
0,20000,Proyecto de investigación interno,2019-01-01,20000,Determinación de la respuesta de gel dosimétri...,2019-11-05 00:00:00,2022-02-28 00:00:00,Facultad de Ciencias,Dpto. de Física,VRI04-19,PONTIFICIA UNIVERSIDAD JAVERIANA,PUJ - Convocatorias,,"""00020394836""","""CC""","""1010169002""","""Andrea Johana Abril Fajardo""",Investigador principal,10.0,120130O0401200,NaN,50000000,0.00,31423755.99,0.0,0,0,NaN
1,20000,Proyecto de investigación interno,2019-01-01,20000,Determinación de la respuesta de gel dosimétri...,2019-11-05 00:00:00,2022-02-28 00:00:00,\N,\N,VRI04-19,PONTIFICIA UNIVERSIDAD JAVERIANA,PUJ - Convocatorias,,\N,\N,\N,"""Daniel Marlow""",Asesor,NaN,120130O0401200,NaN,50000000,0.00,31423755.99,0.0,0,0,NaN
2,20000,Proyecto de investigación interno,2019-01-01,20000,Determinación de la respuesta de gel dosimétri...,2019-11-05 00:00:00,2022-02-28 00:00:00,\N,\N,VRI04-19,PONTIFICIA UNIVERSIDAD JAVERIANA,PUJ - Convocatorias,,\N,\N,\N,"""C\u00e9sar Sierra""",Asesor,NaN,120130O0401200,NaN,50000000,0.00,31423755.99,0.0,0,0,NaN
3,20000,Proyecto de investigación interno,2019-01-01,20000,Determinación de la respuesta de gel dosimétri...,2019-11-05 00:00:00,2022-02-28 00:00:00,\N,\N,VRI04-19,PONTIFICIA UNIVERSIDAD JAVERIANA,PUJ - Convocatorias,,"""00010115666""","""CE""","""353873""","""Luis Agulles Pedros""",Asesor,NaN,120130O0401200,NaN,50000000,0.00,31423755.99,0.0,0,0,NaN
4,20001,Proyecto de investigación interno,2019-01-01,20001,Desarrollo de una herramienta informática para...,2020-04-01 00:00:00,2021-12-15 00:00:00,Fac. de Arquitectura y Diseño,Dpto. de Arquitectura,VRI04-19,PONTIFICIA UNIVERSIDAD JAVERIANA,PUJ - Convocatorias,Calidad y habitabilidad de la vivienda,"""00020313829""","""CC""","""80136645""","""Juan Guillermo Yunda Lozano""",Investigador principal,50.0,120132M0401200,NaN,49709537.13,5380000.04,74499933.39,0.0,0,0,NaN


In [88]:
proyectos.loc[(proyectos["VALOR_APROBADO_PATROCINADOR"]=="\\N"),"VALOR_APROBADO_PATROCINADOR"]=0

KeyError: 'VALOR_APROBADO_PATROCINADOR'

In [74]:
proyectos.loc[(proyectos["VALOR_APROBADO_PATROCINADOR"]=="\\N")]

,ID_PROPUESTA,TIPO_DE_PROPUESTA,FECHA_DE_NEGOCIACION,ID_PROYECTO,TITULO_DEL_PROYECTO,FECHA_INICIAL_REAL,FECHA_FINAL_REAL,NOMBRE_FACULTAD,NOMBRE_DEL_DEPARTAMENTO,CONVOCATORIA,NOMBRE_PATROCINADOR,TIPO_DE_FINANCIADOR,NOMBRE_GRUPO_DE_INVESTIGACION,MIEMBRO_DEL_EQUIPO,TIPO_DE_DOCUMENTO,NUMERO_DE_DOCUMENTO,NOMBRES_Y_APELLIDOS,ROL_EN_EL_PROYECTO,PORCENTAJE_DEDICACION,CODIGO_PRESUPUESTO,CODIGO_CONTRATO_SECRE_JURIDICA,VALOR_APROBADO_PATROCINADOR,CONTRAPARTIDA_TERCEROS,CONTRAPARTIDA_REC__PROPI,CONTRAPARTIDA_REC__NUEVO,REC__NUEVOS_UNIDADE_ACADEMICA,VALOR_CONTRAPARTIDA_TOTAL,ESTADO


In [75]:
proyectos["VALOR_APROBADO_PATROCINADOR"]=proyectos["VALOR_APROBADO_PATROCINADOR"].astype(float)

In [84]:
proyectos["FECHA_DE_NEGOCIACION"]=pd.to_datetime(proyectos["FECHA_DE_NEGOCIACION"],format='mixed')

In [85]:
proyectos["FECHA_DE_NEGOCIACION"][:10]

0   2019-01-01
1   2019-01-01
2   2019-01-01
3   2019-01-01
4   2019-01-01
5   2019-01-01
6   2019-01-01
7   2019-01-01
8   2019-01-01
9   2019-01-01
Name: FECHA_DE_NEGOCIACION, dtype: datetime64[ns]

In [ ]:
proyectos["FECHA_DE_NEGOCIACION"]=pd.to_datetime(proyectos["FECHA_DE_NEGOCIACION"],format='mixed')